<a href="https://colab.research.google.com/github/kpi6research/Text-Style-Transfer-with-Gradient-Steering/blob/master/Gradient_Steering.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### Import delle librerie necessarie 

In [0]:
import sys
sys.path.append('/content/drive/My Drive/Python Files')

import model as m
import Decoding_Strategies
import ModelUtility

import torch
import torch.nn as nn
import torch.optim as optim
import torchtext
from torchtext.data import Field, BucketIterator
from torchtext.datasets import TranslationDataset, Multi30k
import torch.nn.functional as F
  
import spacy
import numpy as np

import random
import math
import time
import queue as Q
import operator

import os
import pandas as pd
import re
import json

### Download Word Vecs

In [2]:
!unzip /content/drive/My\ Drive/wiki-news-300d-1M.vec.zip -d /content/sample_data

Archive:  /content/drive/My Drive/wiki-news-300d-1M.vec.zip
  inflating: /content/sample_data/wiki-news-300d-1M.vec  


In [0]:
import io
def load_vectors(fname):
    fin = io.open(fname, 'r', encoding='utf-8', newline='\n', errors='ignore')
    n, d = map(int, fin.readline().split())
    data = {}
    for line in fin:
        tokens = line.rstrip().split(' ')
        data[tokens[0]] = map(float, tokens[1:])
    return data

embedded_vec = load_vectors("/content/sample_data/wiki-news-300d-1M.vec")

### Inizializzazione librerie e metodo per la tokenizzazione

In [3]:
!pip install -U spacy
#scarichiamo il tokenizer, necessario per rendere frasi liste di tokens
!python -m spacy download en_core_web_sm
#Load del modello dei singoli linguaggi
spacy_en = spacy.load("en_core_web_sm")

#Tokenizer delle frasi in input tramite spacy
def tokenize_en(text):
    """
    Genera token della frase, creando una lista di stringhe
    """
    return [tok.text for tok in spacy_en.tokenizer(text)]

Requirement already up-to-date: spacy in /usr/local/lib/python3.6/dist-packages (2.2.4)
✔ Download and installation successful
You can now load the model via spacy.load('en_core_web_sm')


###Generazione campi torchtext e dataset

Generiamo il Field, il dataset e l'intersezione tra dataset e vocabolario di FastText

In [4]:
#Il campo in cui indichiamo come devono essere processate le frasi
SRC = torchtext.data.Field(tokenize=tokenize_en,
            init_token = '<sos>', 
            eos_token = '<eos>',
            unk_token = 'unk',
            fix_length = 41,
            lower = True)

#Creazione e Split del Dataset,partendo dal file CSV che contiene le frasi, basta codificare solo il SRC nel file
#Tanto SRC==TRG
dataset = torchtext.datasets.TranslationDataset("/content/drive/My Drive/new_tokenized_merged_dataset.csv", exts= ("",""), fields = (SRC,SRC))

SRC.build_vocab(dataset, min_freq=3)
len(SRC.vocab)

def intersection(lst1, lst2): 
    lst3 = [value for value in lst1 if value in lst2] 
    return lst3 

vocab = SRC.vocab.itos
check = list(embedded_vec.keys())

vocab_intersect = intersection(vocab,check)
print(len(vocab_intersect))

17965


Filtriamo il dataset levando tutte le frasi che contengono parole che non sono in FastText

In [5]:
#Threshold per vedere se la frase ha abbastanza embeddings
final_emb = dict()
thresh = 1.0
remove_from_train = []
for example in dataset.examples:
  temp_example = vars(example)['src']
  percent = 0
  for value in temp_example:
    if value in vocab_intersect:
      percent += 1
  percent = percent/len(temp_example)
  if percent < thresh:
    remove_from_train.append(example)
  # else:
  #   for value in temp_example:
  #     final_emb[value] = embedded_vec[value]
      
print(len(remove_from_train))

632


Rimuoviamo le frasi che non contengono le cui parole non fanno tutte parte di FastText e rigeneriamo il vocabolario.

In [6]:
for value in remove_from_train:
  try:
    dataset.examples.remove(value)
  except:
    print(value)

SRC.build_vocab(dataset, min_freq=3)
print("Lunghezza del dizionario:")
print(len(SRC.vocab))
print("Lunghezza del dataset:")
print(len(dataset.examples))

Lunghezza del dizionario:
17809
Lunghezza del dataset:
43970


Generiamo la matrice di embeddings da passare direttamente al modello come layer di Embeddings

In [0]:
special_tokens = ['unk','<pad>','<sos>','<eos>']

for value in SRC.vocab.itos:
  if value not in special_tokens:
      final_emb[value] = embedded_vec[value]
    
final_emb['<eos>'] = embedded_vec['eos']
final_emb['<sos>'] = embedded_vec['sos']
final_emb['unk'] = embedded_vec['unk']

#Impostiamo il l'embedding di pad
pad = [0]*300
pad[0] = 1
final_emb['<pad>'] = pad

#Spacchettiamo le maps
for key in final_emb.keys():
  final_emb[key] = list(final_emb[key])

#Prendo i pesi ordinati rispetto al dizionario, in questo
#Modo l'ordine si conserva, necessario al corretto
#Funzionamento degli embeddings.
emb_list = []
for value in SRC.vocab.itos:
  emb_list.append(final_emb[value])

#Porto i pesi in numpy_array
emb_weigths = np.array(emb_list, dtype = float)

#Salvo <sos> per usarlo come input dell'attn
sos_emb = final_emb['<sos>']

Generiamo Train, Test e Validation Data e successivamente liberiamo la RAM

In [0]:
#Split del train, validation e test
train_data, valid_data, test_data = dataset.split(split_ratio=[0.7,0.15,0.15])

BATCH_SIZE = 64

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

train_iterator, valid_iterator, test_iterator = torchtext.data.BucketIterator.splits(
    (train_data, valid_data, test_data), 
    batch_size = BATCH_SIZE, 
    device = device)

#Cancello queste variabili liberando circa 19GB di RAM
del(final_emb)
del(check)
del(embedded_vec)
del(vocab_intersect)#Parametri della rete

###Creazione matrice One Hot per la BoW

In [0]:
df = pd.read_fwf("/content/drive/My Drive/wordlists/bow_economy.txt")
bow = []
for value in df['business']:
  bow.append(value)

bow.append('business')

indexes = []
for value in bow:
  indexes.append(SRC.vocab.stoi[value])

one_hot = torch.zeros(len(indexes),len(SRC.vocab.itos)).cuda()

word_index = 0
for value in indexes:
  one_hot[word_index,value] = 1  
  word_index +=1

###Modello e inizializzazione del modello

In [0]:
#Encoder, con definizione dell'architettura e del forward
class Encoder(nn.Module):
    def __init__(self,input_dim, dim, n_layers, dropout,emb_weights, device):
        
        super().__init__()
        self.device = device
        self.dim = dim
        self.n_layers = n_layers
        self.input_dim = input_dim
        self.emb_weights = emb_weights

        self.embedding = nn.Embedding(input_dim, dim)
        self.embedding.weight = nn.Parameter(torch.from_numpy(emb_weights).to(torch.float))
        self.embedding.weight.requires_grad= False

        self.rnn = nn.LSTM(dim, dim, n_layers, dropout = dropout, batch_first=True)
        
        
    def forward(self, src):      
 
        #src = [time_step,batch_size]
        src = torch.t(src)
        embedded = self.embedding(src)
        # print("Shape embedding size encoder:")
        # print(embedded.shape) # shape = (BS,TS,HDIM)
        output, (hidden, cell) = self.rnn(embedded)

        return hidden, cell, output



class Attention(nn.Module):
  def __init__(self, dim, att_dim,device,heads = 3):
    
    super().__init__()

    self.dim = dim #Dimensione sia degli embedding che degli hidden_states
    self.att_dim = att_dim #Lunghezza massima della stringa, necessaria alla creazione dello strato attn
    self.device = device #GPU
    self.heads = heads #N di attention
    
    #Prima matrice dei pesi dell'attention
    self.w1 = torch.zeros(2 * self.dim , 
                          self.att_dim, 
                          device = device, 
                          requires_grad = True)
    self.w1 = torch.nn.Parameter(self.w1)
    
    #Seconda matrice dell'attention
    self.w2 = torch.zeros(self.att_dim, 
                          self.heads,
                          device = device,
                          requires_grad = True) 
    self.w2 = torch.nn.Parameter(self.w2)
 
  def forward(self, encoder_outputs, prev_dec_output):
    
    #print(prev_dec_output.shape) # shape = (BS, 1(TS dei layer della rete), 300)
    
    #print(encoder_outputs.shape) # shape = (BS,TS,HDIM)

    #Aumentiamo le dimensioni così da permettere il concat
    prev_dec_output_expand = prev_dec_output.expand_as(encoder_outputs) # shape = (BS,TS,HDIM)

    #Concateniamo l'econder_outputs con il precedente stato del decoder
    encoder_outputs_concat = torch.cat((encoder_outputs, prev_dec_output_expand), 2) # shape = (BS, TS, 2*HDIM)

    #Primo MatMul
    att_op_1 = torch.matmul(encoder_outputs_concat, self.w1) # shape = (BS, TS, ATT_DIM)
    att_op_1 = torch.tanh(att_op_1)

    #Secondo MatMul
    att_op_2 = torch.matmul(att_op_1, self.w2)  # shape = (BS, TS, HEADS)
    att_op_2 = torch.tanh(att_op_2)
    #Softmax sulla seconda dimesione della matrice di attention
    att_weights = F.softmax(att_op_2,dim=1) #shape = (BS, TS, HEADS)
    
    #print(att_weights)
    #Trasposta della matrice degli output dell'encoder
    encoder_outputs_concat = encoder_outputs_concat.permute(0,2,1) # shape = (BS,2*HDIM,TS)
    
    #Terzo MatMul, tra encoder_outputs concatenato con il prev_dec_state e gli attn_weights
    att_output = torch.matmul(encoder_outputs_concat, att_weights) # shape = (BS,2*HDIM,HEADS)
    att_output = torch.tanh(att_output)
    att_output = att_output.reshape(att_output.shape[0],att_output.shape[1]*att_output.shape[2]) # shape =(BS,2*HDIM*HEADS)
    
    return att_output


class Decoder(nn.Module):
  def __init__(self, dim, output_dim, dropout, n_layers,device,emb_weights,heads = 3):
    super().__init__()

    self.device = device #GPU
    self.dim = dim #Dimensione sia degli embedding che degli hidden_states
    self.output_dim = output_dim #Dimensione del vocabolario di arriva
    self.n_layers = n_layers #numero di layer dell'encoder
    self.emb_weights = emb_weights #Pesi degli embeddings 

    self.w3 = torch.zeros(self.dim, self.output_dim, 
                          device = device, 
                          requires_grad = True) #Terza matrice dell'attention
    self.w3 = torch.nn.Parameter(self.w3)

    self.w4 = torch.zeros(self.dim * 6 + self.dim,
                          self.dim, 
                          device = device, 
                          requires_grad = True) #Terza matrice dell'attention
    self.w4 = torch.nn.Parameter(self.w4)

    self.rnn = nn.LSTM(self.dim,
                       self.dim, 
                       n_layers, dropout = dropout, 
                       batch_first = True) #LSTM

    self.embedding = nn.Embedding(output_dim, dim)
    self.embedding.weight = nn.Parameter(torch.from_numpy(emb_weights).to(torch.float))
    self.embedding.weight.requires_grad= False

  def forward(self,att_output, input,hidden, cell):
    
    #Da implementare embeddings
    emb = self.embedding(input)
    #Nel caso della Beam Search abbiamo meno dimensioni
    try:
      if (len(emb.shape) == 1):
        decoder_input = torch.cat((att_output, emb.unsqueeze(0)),1).unsqueeze(0) # shape = (1,BS,2*HDIM*HEADS + HDIM)
      else:
        decoder_input = torch.cat((att_output, emb),1).unsqueeze(0) # shape = (1,BS,2*HDIM*HEADS + HDIM)
    except:
      print(att_output.shape)
      print(emb.shape)
    decoder_input = decoder_input.permute(1,0,2) # shape = (BS, 1, 2*HDIM*HEADS + HDIM)
    
    decoder_input_w4 = torch.matmul(decoder_input,self.w4) # shape = (BS, 1, HDIM)
    #decoder_input_w4 = torch.tanh(decoder_input_w4)

    dec_state, (hidden, cell) = self.rnn(decoder_input_w4,(hidden,cell))

    output = torch.matmul(dec_state,self.w3)

    return output.squeeze(1), hidden, cell, dec_state

class Seq2Seq(nn.Module):
    def __init__(self, encoder, decoder, attention, device, sos_emb,max_length, beam_width = 3):
        super().__init__()
        self.encoder = encoder
        self.decoder = decoder
        self.attention = attention
        self.device = device
        self.sos_emb = sos_emb
        self.max_length = max_length
        self.beam_width = beam_width
        assert encoder.dim == decoder.dim, \
            "Hidden dimensions of encoder and decoder must be equal!"
        assert encoder.n_layers == decoder.n_layers, \
            "Encoder and decoder must have equal number of layers!"

#Dimensione del dizionario dell'input
#Dimensione del dizionario di output
#Dimensione interna, sia degli embeddings che degli hidden states
#Numero di layers delle LSTM dell'Encoder e del Decoder
#Probabiltà di Dropout delle LSTM
#Grandezza massima delle stringhe in entrata(N di token massimi)
#Matrice degli Embeddings
#Device su cui far runnare tutto, si spera GPU
def model_initialization(INPUT_DIM,OUTPUT_DIM,DIMENSION,N_LAYERS, DROPOUT, MAX_SIZE,emb_weights, device,sos_emb):
    #Dichiarazione del modello
    enc = Encoder(INPUT_DIM, DIMENSION, N_LAYERS, DROPOUT,emb_weights,device)
    attn = Attention(DIMENSION, MAX_SIZE, device)
    dec = Decoder(DIMENSION,OUTPUT_DIM,DROPOUT,N_LAYERS,device,emb_weights)
    model = Seq2Seq(enc, dec, attn, device, sos_emb, MAX_SIZE).to(device)
   
    model.cuda()

    for name, param in model.named_parameters():
        nn.init.uniform_(param.data, -0.09, 0.09)

    return model


Per adesso non utilizziamo criteri di ottimizzazione (Nel nostro caso CrossEntropyLoss)

In [11]:
#Input e output sono due OHE sui vocabolari di partenza e di arrivo
INPUT_DIM = len(SRC.vocab)
OUTPUT_DIM = INPUT_DIM
#Dimensione dello strato di embedding, anche se io dovrei usare fastText

#Dimensione degli strati nascosti, uguale per ENC|DEC, vale anche per gli embeddings
DIMENSION = 300
N_LAYERS = 1

#Probabilità del dropout, applicato sullo strato di embedding
DROPOUT = 0.5
modello = model_initialization(INPUT_DIM,OUTPUT_DIM,DIMENSION, N_LAYERS,DROPOUT,41,emb_weigths,device,sos_emb)

#Definiamo il criterio di Loss, controllando di indicare su quali elementi non applicarlo, in questo caso il padding
SRC_PAD_IDX = SRC.vocab.stoi[SRC.pad_token]


#criterion = nn.CrossEntropyLoss(ignore_index = SRC_PAD_IDX)

def count_parameters(modello):
    return sum(p.numel() for p in modello.parameters() if p.requires_grad)

print(f'The model has {count_parameters(modello):,} trainable parameters')

def epoch_time(start_time, end_time):
    elapsed_time = end_time - start_time
    elapsed_mins = int(elapsed_time / 60)
    elapsed_secs = int(elapsed_time - (elapsed_mins * 60))
    return elapsed_mins, elapsed_secs

#Metodo di ottimizzazione dei parametri della rete
optimizer = optim.Adam(modello.parameters(), lr = 0.001)

The model has 7,442,223 trainable parameters


/usr/local/lib/python3.6/dist-packages/torch/nn/modules/rnn.py:50: UserWarning: dropout option adds dropout after all but last recurrent layer, so non-zero dropout expects num_layers greater than 1, but got dropout=0.5 and num_layers=1
  "num_layers={}".format(dropout, num_layers))


In [12]:
modello.load_state_dict(torch.load("/content/drive/My Drive/new_model_weigths_13_04.pt"))

<All keys matched successfully>

In [0]:
#Blocchiamo tutti i parametri del modello, tanto è già addestrato
for params in modello.parameters():
  params.requires_grad = False

###Metodi per la perturbazione della predizione

Metodo che ritorna una variabile trainable utilizzando i dati dell'input

In [0]:
from torch.autograd import Variable

def to_var(x, requires_grad=False, volatile=False):
    if torch.cuda.is_available():
        x = x.cuda()
    return Variable(x, requires_grad=requires_grad, volatile=volatile)

Primo metodo della perturbazione, cicla su tutto l'iteratore e stampa frase perturbata e frase target

In [0]:
import time

#Ritorna una lista di frasi perturbate, prese dall'iterator
def generate_perturbed_prediction(model,iterator,SRC,one_hot):

      # Per ogni batch nell'iteratore
      for i, batch in enumerate(test_iterator):
        src = batch.src
        trg = batch.trg

        #Per ogni frase nella batch
        for j in range(len(src)):
          
          enc_input = src[:,j].unsqueeze(0)
          output_perturb = perturb_phrase(model, enc_input,one_hot)
          
          #Stampiamo la frase originale
          phrase = ""
          print("Target phrase")
          for val in enc_input[0]:
            if(val != 3 and val != 1):
              phrase +=  " " + SRC.vocab.itos[val]
          print(phrase)
          
          #Stampiamo la frase perturbata
          phrase = ""
          print("Phrase predicted:")
          for val in output_perturb:
            if(val != 3 and val != 1):
              phrase +=  " " + SRC.vocab.itos[val]
          print(phrase)
          print(" ")
          
          #Giusto per vedere cosa stampa sennò stampa troppo velocemente
          time.sleep(1)

      return;

Metodo che ritorna la frase perturbata, prendendo in input una frase, il modello e la BoW

In [0]:
#Ritorna la frase perturbata
def perturb_phrase(model,enc_input,one_hot):
  #Lista delle predizioni perturbate
  output = []

  #Frase in input
  enc_input = torch.t(enc_input)

  #hidden shape = torch.Size([1, 41, 300])
  #cell shape = torch.Size([1, 41, 300])
  #encoder_output shape = torch.Size([1, 41, 300])
  hidden, cell, encoder_outputs = model.encoder(enc_input)
  
  #Piccola constante necessaria
  small_const = 1e-15
  sos_emb_matrix = torch.tensor(model.sos_emb,device=model.device)
  sos_emb_matrix = sos_emb_matrix.expand_as(torch.zeros(1,1, 300))

  #Generiamo gli input del decoder al passo 0
  dec_state_t = sos_emb_matrix
  dec_input = enc_input[0,0]
  output.append(dec_input.item())

  #Layer di Attention
  att_output_original = model.attention(encoder_outputs, dec_state_t)
  output_original, hidden_original, cell_original, dec_state_original = model.decoder(att_output_original,
                                                                                          dec_input, 
                                                                                          hidden, cell)
  #Riassegniamo tutte le componenti per il prossimo timestep 
  hidden = hidden_original
  cell = cell_original
  dec_state_t = dec_state_original 
  dec_input = output_original.argmax(1)
  output.append(dec_input[0].item())

  for i in range(model.max_length - 1):
      #Questi sono gli originali, ad ogni passo li andremo a modificare 
      #all'interno di questo ciclo

      #Layer di Attention
      att_output_original = model.attention(encoder_outputs, dec_state_t)

      #Layer di Decoding
      #hidden shape = [1,1,300]
      #cell shape = [1,1,300]
      output_original, hidden_original, cell_original, dec_state_original = model.decoder(att_output_original,
                                                                                          dec_input, 
                                                                                          hidden, cell)
      #primi stati perturbati
      perturbed_hidden = hidden.detach().clone()
      perturbed_dec_state = dec_state_t.detach().clone()
      perturbed_encoder_outputs = encoder_outputs.detach().clone()

      #Genero variabili trainable
      perturbed_hidden = to_var(perturbed_hidden,True)
      perturbed_dec_state = to_var(perturbed_dec_state,True)
      perturbed_encoder_outputs = to_var(perturbed_encoder_outputs, True)

      #Calcoliamo gli hidden state e i dec state perturbati
      perturbed_hidden, perturbed_dec_state, perturbed_encoder_outputs = perturb_time_step(perturbed_hidden,
                                                                perturbed_dec_state, 
                                                                model,
                                                                one_hot,
                                                                perturbed_encoder_outputs,
                                                                dec_input,
                                                                cell,
                                                                output_original,
                                                                small_const
                                                                )
      
      #Calcoliamo l'output finale perturbato 
      perturbed_att_output = model.attention(perturbed_encoder_outputs, perturbed_dec_state)

      perturbed_output,_,_,_ = model.decoder(perturbed_att_output,
                                             dec_input,
                                             hidden,cell)
      
      #Calcoliamo le probabilità dei due modelli, per fare successivamente il blend
      original_probs = F.softmax(output_original, dim = 1)
      perturbed_probs = F.softmax(perturbed_output, dim = 1)
      top1_perturb = perturbed_probs.argmax(1)

      #Blend delle probabilità tramite post norm geometric mean fusion
      gm_scale = 0.9
      final_probs = (perturbed_probs ** gm_scale) * (original_probs ** (1 - gm_scale))

      #Rinormalizziamolo nel caso in cui non lo sia dopo il blend
      if(torch.sum(final_probs) <= 1):
        final_probs = final_probs / torch.sum(final_probs)

      #Riassegniamo tutte le componenti per il prossimo timestep 
      hidden = hidden_original
      cell = cell_original
      dec_state_t = dec_state_original 
      dec_input = final_probs.argmax(1)
      # if(top1_perturb.item() != 3):
      #   print(top1_perturb.item())
      #   print(dec_input[0].item())
      output.append(dec_input[0].item())

  return output

Metodo che perturba un singolo TimeStep e ritorna l'hidden_state, il decoder_state e l'encoder_output perturbati per un certo numero di TimeStep

In [0]:
def perturb_time_step(perturbed_hidden, 
                      perturbed_dec_state, 
                      model,
                      one_hot,
                      perturbed_encoder_outputs,
                      dec_input,
                      cell,
                      output_original,
                      small_const):
  
  #Genero variabili trainable
  perturbed_hidden = to_var(perturbed_hidden,True)
  perturbed_dec_state = to_var(perturbed_dec_state,True)
  
  #Inizializziamo il le matrici che conterranno i gradienti
  #Ad ogni passo sommiamo allo stesso hidden un gradiente accumulato 
  hidden_grad_sum = torch.zeros(1,1, device= model.device).expand_as(perturbed_hidden).clone()
  dec_state_grad_sum = torch.zeros(1,1, device = model.device).expand_as(perturbed_dec_state).clone()
  enc_output_grad_sum = torch.zeros(1,1, device = model.device).expand_as(perturbed_encoder_outputs).clone()

  # print("****************************")

  #Per ogni passo dell'interazione
  for i in range(10):
    #Sommiamo le perturbazioni agli hidden
    perturbed_hidden.data += hidden_grad_sum
    perturbed_dec_state.data += dec_state_grad_sum
    perturbed_encoder_outputs.data += enc_output_grad_sum
    
    loss = 0.0

    #Generiamo gli output del modello perturbato
    att_output_perturbed = model.attention(perturbed_encoder_outputs, perturbed_dec_state)
    output_perturbed,_,_,_ = model.decoder(att_output_perturbed, dec_input, perturbed_hidden, cell)
    
    #Softmax per farlo diventare una vera distribuzione
    probabs_perturbed = F.softmax(output_perturbed,dim = 1)
    #Generazione BOW Loss
    #Cancella le parole nella bow certe volte
    #Altre volte non si sa perchè la somma fa un numero che è quasi 1, ma non è possibile
    good_logits = torch.mm(probabs_perturbed,torch.t(one_hot))
    good_logits = good_logits.clone()
    loss_word = torch.sum(good_logits.squeeze(0)).clone()
    #Per adesso mettiamo un "cerotto"
    if(loss_word.item() >= 0.9):
      loss_word = 0.0
      # print("Ricalcolo la somma")
      #Da capire perchè la BOW si bugga, dato che ci sono certe volte degli elementi troppo grandi
      for value in good_logits.squeeze(0):
        
        if value < 0.9:
          loss_word += value
      # print("Somma ricalcolata")
      # print(loss_word)
    #print(loss_word)
    # if(probabs_perturbed.argmax(1).item() != 3):
    #   print(" ")
    #   print("Somma delle probabilità della bow")
    #   print(loss_word)
    loss_word = torch.log(loss_word)
    #Sommiamo la bow_loss alla loss totale
    loss += loss_word

    #Calcolo della KL 
    kl_scale = 0.01
    p_original = F.softmax(output_original, dim = 1)
    p_original = p_original + small_const * (p_original <= small_const).type(torch.FloatTensor).cuda().detach()
    correction = small_const * (probabs_perturbed <= small_const).type(torch.FloatTensor).cuda().detach()
    corrected_probabs = probabs_perturbed + correction.detach()
    kl_loss = kl_scale * ((corrected_probabs * (corrected_probabs / p_original).log()).sum())
    # if(probabs_perturbed.argmax(1).item() != 3):
    #   print("Parola predetta:")
    #   print(probabs_perturbed.argmax(1).item())
    #   print("BOW Loss")
    #   print(loss_word)
    loss -= kl_loss

    #Passo di backward
    loss.backward()
    
    #Gradiente normalizzato
    g_n_hidden_state = torch.norm(perturbed_hidden.grad) + small_const
    g_n_dec_state = torch.norm(perturbed_dec_state.grad) + small_const
    g_n_enc_output = torch.norm(perturbed_encoder_outputs) + small_const
    
    #Inizializzazione dei parametri della rete
    stepsize = 0.02
    gamma = 1.5

    #Aggiorniamo il delta t
    hidden_grad_sum += stepsize * (perturbed_hidden.grad / g_n_hidden_state ** gamma) 
    dec_state_grad_sum += stepsize * (perturbed_dec_state.grad / g_n_dec_state ** gamma)
    enc_output_grad_sum += stepsize * (perturbed_encoder_outputs.grad / g_n_enc_output ** gamma)

    #Azzeriamo il gradiente di ogni variabile
    perturbed_hidden.grad.data.zero_()
    perturbed_dec_state.grad.data.zero_()
    perturbed_encoder_outputs.grad.data.zero_()

    #Nel dubbio, detach delle matrici 
    perturbed_hidden = perturbed_hidden.detach()
    perturbed_dec_state = perturbed_dec_state.detach()
    perturbed_encoder_outputs = perturbed_encoder_outputs.detach()

    #Genero variabili trainable
    perturbed_hidden = to_var(perturbed_hidden,True)
    perturbed_dec_state = to_var(perturbed_dec_state,True)
    perturbed_encoder_outputs = to_var(perturbed_encoder_outputs,True)

  #Torniamo i stati perturbati
  perturbed_hidden.data += hidden_grad_sum
  perturbed_dec_state.data += dec_state_grad_sum
  perturbed_encoder_outputs.data += enc_output_grad_sum

  return perturbed_hidden, perturbed_dec_state, perturbed_encoder_outputs

Trovato bug su sum, spesso somma prendendo direttamente i valori da tutta la distribuzione piuttosto che dalla BoW, infatti la somma viene 1 o 0.999, il che non è possibile se prendo una partizione di un gruppo normalizzato.

Questo si presenta spesso se la parola in a quel TS nel SRC è una parola della BoW.

Per adesso ci ho messo un cerotto: I valori troppo grandi vengono rimossi dalla somma, da capire come risolvere al meglio.


In [35]:
generate_perturbed_prediction(modello,test_iterator,SRC,one_hot)

Target phrase
 <sos> hard cases make bad law ...
Phrase predicted:
 <sos> hard cases make bad law ... just
 
Target phrase
 <sos> power of money and economy !
Phrase predicted:
 <sos> power of money and economy !
 
Target phrase
 <sos> 's cruise cock - up via
Phrase predicted:
 <sos> 's working isolation - up via
 
Target phrase
 <sos> craziest alert i ever got !
Phrase predicted:
 <sos> craziest alert i ever got !
 
Target phrase
 <sos> 6.6 million americans file jobless claims
Phrase predicted:
 <sos> 6.6 million americans file jobless claims
 
Target phrase
 <sos> global bailouts to fight the -
Phrase predicted:
 <sos> global manufacturers to fight the -
 
Target phrase
 <sos> i just be minding my business
Phrase predicted:
 <sos> i just be minding my business
 
Target phrase
 <sos> it 's a lonely beach !
Phrase predicted:
 <sos> it 's a lonely beach !
 
Target phrase
 <sos> mines close as tightens controls -19
Phrase predicted:
 <sos> tools close as stories free concerns just
 
Tar

KeyboardInterrupt: ignored

In [0]:
utils = ModelUtility.ModelUtility()
utils.predict(modello,test_iterator, SRC)